In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import os.path
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
import csv


warnings.simplefilter(action='ignore', category = FutureWarning)
%matplotlib inline

physical_dev = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs avail: ", len(physical_dev))
tf.config.experimental.set_memory_growth(physical_dev[0], True)
#tf.debugging.set_log_device_placement(True)


#Data org
os.chdir('C:/venv/tensorflow-scripting/data/lung-scans')

if os.path.isdir('train/healthy') is False:
    os.makedirs('train/healthy')
    os.makedirs('train/bacterial')
    os.makedirs('train/viral')
    os.makedirs('valid/healthy')
    os.makedirs('valid/bacterial')
    os.makedirs('valid/viral')
    os.makedirs('test/healthy')
    os.makedirs('test/bacterial')
    os.makedirs('test/viral')

    

#Weirdness with rel path
CURR_DIR = os.getcwd()
print(CURR_DIR)

'''
class_id = 0 if the image corresponds to a subject without disease (normal)

class_id = 1 if the image corresponds to a patient with bacterial pneumonia

class_id = 2 if the image corresponds to a patient with viral pneumonia
'''

with open('labels/labels_train.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
#HEALTHY
        if(row['class_id'] == '0'):
            if(len([name for name in os.listdir(CURR_DIR + '/train/healthy/')
                if os.path.isfile(os.path.join(CURR_DIR + '/train/healthy/', name))]) < 800):
                file = '/train_images/' + row['file_name']
                shutil.copy(CURR_DIR + file, CURR_DIR + '/train/healthy/')
            elif(len([name for name in os.listdir(CURR_DIR + '/valid/healthy/')
                if os.path.isfile(os.path.join(CURR_DIR + '/valid/healthy/', name))]) < 100):
                file = '/train_images/' + row['file_name']
                shutil.copy(CURR_DIR + file, CURR_DIR + '/valid/healthy/')
            elif(len([name for name in os.listdir(CURR_DIR + '/test/healthy/')
                if os.path.isfile(os.path.join(CURR_DIR + '/test/healthy/', name))]) < 50):
                file = '/train_images/' + row['file_name']
                shutil.copy(CURR_DIR + file, CURR_DIR + '/test/healthy/')
#BACTERIAL
        if(row['class_id'] == '1'):
            if(len([name for name in os.listdir(CURR_DIR + '/train/bacterial/')
                if os.path.isfile(os.path.join(CURR_DIR + '/train/bacterial/', name))]) < 800):
                file = '/train_images/' + row['file_name']
                shutil.copy(CURR_DIR + file, CURR_DIR + '/train/bacterial/')
            elif(len([name for name in os.listdir(CURR_DIR + '/valid/bacterial/')
                if os.path.isfile(os.path.join(CURR_DIR + '/valid/bacterial/', name))]) < 100):
                file = '/train_images/' + row['file_name']
                shutil.copy(CURR_DIR + file, CURR_DIR + '/valid/bacterial/')
            elif(len([name for name in os.listdir(CURR_DIR + '/test/bacterial/')
                if os.path.isfile(os.path.join(CURR_DIR + '/test/bacterial/', name))]) < 50):
                file = '/train_images/' + row['file_name']
                shutil.copy(CURR_DIR + file, CURR_DIR + '/test/bacterial/')
#VIRAL                        
        if(row['class_id'] == '2'):
            if(len([name for name in os.listdir(CURR_DIR + '/train/viral/')
                if os.path.isfile(os.path.join(CURR_DIR + '/train/viral/', name))]) < 800):
                file = '/train_images/' + row['file_name']
                shutil.copy(CURR_DIR + file, CURR_DIR + '/train/viral/')
            elif(len([name for name in os.listdir(CURR_DIR + '/valid/viral/')
                if os.path.isfile(os.path.join(CURR_DIR + '/valid/viral/', name))]) < 100):
                file = '/train_images/' + row['file_name']
                shutil.copy(CURR_DIR + file, CURR_DIR + '/valid/viral/')
            elif(len([name for name in os.listdir(CURR_DIR + '/test/viral/')
                if os.path.isfile(os.path.join(CURR_DIR + '/test/viral/', name))]) < 50):
                file = '/train_images/' + row['file_name']
                shutil.copy(CURR_DIR + file, CURR_DIR + '/test/viral/')                   

os.chdir('../../')

'''   
train_path = 'data/dogs-and-cats/train'
valid_path = 'data/dogs-and-cats/valid'
test_path = 'data/dogs-and-cats/test'
'''


Num GPUs avail:  1
C:\venv\tensorflow-scripting\data\lung-scans
